# **Telecom Churn Analytics:** 
## ***Building Predictive Models to Improve Customer Retention***

### Section 2 - ETL 🧹🛠️

---

### Objectives

In this section, the aim is to prepare a cleaned dataset for visualization and analysis from the raw data files. There are ETL procedures form data extraction, data cleaning and processing to data load.


### Inputs

* Datasets used for this analysis is the retail data set from Kaggle (https://www.kaggle.com/datasets/mubeenshehzadi/customer-churn-dataset/). 

* 1 raw file will be used.
    * [telecom_customer_churn.csv](../dataset/raw/telecom_customer_churn.csv) 

### Outputs

* A cleaned dataset will be save as a CSV file below
    * [telecom_customer_churn_cleaned.csv](../dataset/processed/telecom_customer_churn_cleaned
    .csv)





---

### Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

In [1]:
import os

os.chdir(os.path.dirname(os.getcwd()))
current_dir = os.getcwd()
print(f"You set a new current directory: {current_dir}")


You set a new current directory: /Users/denniskwok/Documents/data-analytics/telecom-churn-ml-prediction


---

## **Part A**

### **Data Extraction**

In [2]:
import numpy as np
import pandas as pd

#### Step 1: Load Dataset

In [3]:
# Load dataset from csv file
def load_csv(filepath):
    try:
        df = pd.read_csv(filepath)
        print(f"Loaded {filepath} successfully.")
        return df
    except FileNotFoundError:
        print(f"File not found: {filepath}")
        return pd.DataFrame()

df = load_csv("dataset/raw/telecom_customer_churn.csv")


Loaded dataset/raw/telecom_customer_churn.csv successfully.


### Step 2: Overview The RAW Dataset

**General dataframe information** 

In [4]:
df.info() # Display dataframe information  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


**Dataframe data overview**

In [5]:
df.head() # Display the first few rows of the dataframe

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


**Checking missing values**

In [6]:
df.isnull().sum() # Check for missing values

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

---

## **Part B**

### **Data Transformation** 
### *Data cleaning and processing pipeline*

### 🧠 Pipeline Summary: Telecom Churn Data Preprocessing & Feature Engineering

The preprocessing pipeline is designed to transform the raw Telecom Customer Churn dataset into a clean, machine-learning-ready format. It standardises data quality, engineers meaningful service-based features, and applies robust encoding and scaling methods to support predictive modelling.

**Steps in the ETL Pipeline:**

1️⃣ Data Loading & Cleaning

* Source: dataset/raw/Telecom_Customer_Churn.csv

* Removes personally identifiable information (customerID).

* Converts TotalCharges to numeric and imputes missing values using
TotalCharges = MonthlyCharges × tenure.

* Ensures the dataset contains no missing values before model training.

2️⃣ Feature Engineering

* Creates a new CustomerType feature categorising users as: "Phone only", "Internet only", "Both"

* Normalises inconsistent entries (“No internet service” → “No”).

* Derives NumInternetServices, counting the number of active internet-based add-ons such as
OnlineSecurity, OnlineBackup, TechSupport, StreamingTV, etc. This feature quantifies customer engagement with telecom products.

3️⃣ Preprocessing Pipelines

🔹 Numeric Pipeline

* Features: tenure, MonthlyCharges, TotalCharges, NumInternetServices

    Steps:

    * SimpleImputer(strategy='median') – handles any residual missing numeric values.

    * StandardScaler() – standardises features to mean = 0, std = 1 for stable model performance.

🔹 Categorical Pipeline

*   Features: all non-numeric columns (e.g., Contract, PaymentMethod, Gender, CustomerType, etc.)

    Steps:

    * SimpleImputer(strategy='most_frequent') – fills missing categorical values.

    * OneHotEncoder(handle_unknown='ignore', drop='first') – converts categories into binary variables, avoiding multicollinearity.

    Both pipelines are combined via a ColumnTransformer into a single unified preprocessing step.

4️⃣ Outputs & Diagnostics

* The pipeline produces a fully encoded numeric feature matrix ready for model training.

* Prints detailed diagnostics:

    * Number of categorical and numeric features.

    * Mapping of categories per feature.

    * Final encoded column names.

* Asserts that no missing values remain in the final dataset.

In [17]:
"""
ETL & Feature Engineering Pipeline for Telecom Churn Dataset
DataFrame-based implementation (avoids NumPy array output)
"""

import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer

# =========================
# 1. Feature Engineering
# =========================
def add_service_features(df):
    """Add derived service-based features to the dataset."""

    # Define customer type based on available services
    def service_type(row):
        if row['InternetService'] == 'No':
            return "Phone only"
        elif row['PhoneService'] == 'No':
            return "Internet only"
        else:
            return "Both"

    df['CustomerType'] = df.apply(service_type, axis=1)

    # Define internet add-ons
    internet_add_ons = [
        "OnlineSecurity", "OnlineBackup", "DeviceProtection",
        "TechSupport", "StreamingTV", "StreamingMovies"
    ]

    # Normalize categorical values
    for col in internet_add_ons:
        df[col] = df[col].replace("No internet service", "No")

    # Create feature: number of "Yes" internet services
    df["NumInternetServices"] = (df[internet_add_ons] == "Yes").sum(axis=1)

    return df


# =========================
# 2. Data Cleaning
# =========================
def etl_clean_data(df):
    # Add service-based features
    df = add_service_features(df)

    # Drop PII
    if 'customerID' in df.columns:
        df.drop("customerID", axis=1, inplace=True)

    # Convert TotalCharges to numeric
    df["TotalCharges"] = pd.to_numeric(df["TotalCharges"], errors="coerce")

    # Impute missing TotalCharges using MonthlyCharges * tenure
    df.loc[df['TotalCharges'].isnull(), 'TotalCharges'] = (
        df['MonthlyCharges'] * df['tenure']
    )

    return df


# =========================
# 3. Manual Preprocessing (keeping Pandas format)
# =========================
def preprocess_dataframe(df):
    """Perform imputation, scaling, and encoding directly in pandas."""
    
    # Separate numeric and categorical columns
    numeric_features = ["tenure", "MonthlyCharges", "TotalCharges", "NumInternetServices"]
    categorical_features = df.select_dtypes(include=["object"]).columns.tolist()

    # 1️⃣ Impute missing numeric values
    for col in numeric_features:
        df[col] = df[col].fillna(df[col].median())

    # 2️⃣ Scale numeric values (z-score normalization)
    df[numeric_features] = df[numeric_features].apply(
        lambda x: (x - x.mean()) / x.std()
    )

    # 3️⃣ Impute missing categorical values
    for col in categorical_features:
        df[col] = df[col].fillna(df[col].mode()[0])

    # 4️⃣ One-hot encode categorical variables (drop_first=True for multicollinearity)
    df_encoded = pd.get_dummies(df, columns=categorical_features, drop_first=True)

    return df_encoded


# =========================
# 4. Execute ETL + Preprocessing
# =========================
df_cleaned = etl_clean_data(df)
df_encoded = preprocess_dataframe(df_cleaned)

# Save processed dataset
df_cleaned.to_csv("dataset/processed/telecom_customer_churn_cleaned.csv", index=False)
df_encoded.to_csv("dataset/processed/telecom_customer_churn_encoded.csv", index=False)


print("✅ ETL + preprocessing complete!")
print(f"Processed dataset shape: {df_encoded.shape}")
print("Columns:", df_encoded.columns.tolist()[:10], "...")
print(f"Data shape after cleaning: {df_cleaned.shape}")
print(f"Categorical features: {len(df_cleaned.select_dtypes(include=['object']).columns)}")
print(f"Numeric features: {len(df_cleaned.select_dtypes(exclude=['object']).columns)}")
print(f"Data shape after encoding: {df_encoded.shape}")
print('\nNew Columns after One-Hot Encoding:\n')
print(df_encoded.columns.tolist(), '\n')

# Verify no missing values remain
assert df_cleaned.isnull().sum().sum() == 0, "Missing values remain in the DataFrame"




✅ ETL + preprocessing complete!
Processed dataset shape: (7043, 28)
Columns: ['SeniorCitizen', 'tenure', 'MonthlyCharges', 'TotalCharges', 'NumInternetServices', 'gender_Male', 'Partner_Yes', 'Dependents_Yes', 'PhoneService_Yes', 'MultipleLines_No phone service'] ...
Data shape after cleaning: (7043, 22)
Categorical features: 17
Numeric features: 5
Data shape after encoding: (7043, 28)

New Columns after One-Hot Encoding:

['SeniorCitizen', 'tenure', 'MonthlyCharges', 'TotalCharges', 'NumInternetServices', 'gender_Male', 'Partner_Yes', 'Dependents_Yes', 'PhoneService_Yes', 'MultipleLines_No phone service', 'MultipleLines_Yes', 'InternetService_Fiber optic', 'InternetService_No', 'OnlineSecurity_Yes', 'OnlineBackup_Yes', 'DeviceProtection_Yes', 'TechSupport_Yes', 'StreamingTV_Yes', 'StreamingMovies_Yes', 'Contract_One year', 'Contract_Two year', 'PaperlessBilling_Yes', 'PaymentMethod_Credit card (automatic)', 'PaymentMethod_Electronic check', 'PaymentMethod_Mailed check', 'Churn_Yes', '

Remarks: After changing the data type, empty strings or invalid entries become NaN in TotolCharges column. the missing values of TotalCharges was imputed the formula assumes TotalCharges should roughly equal monthly charges multiplied by tenure.

---

## **Part C**

### **Data Load** 
### *Creating cleaned and encoded dataset*

* The resulting preprocessor in the pipeline can be directly plugged into an end-to-end machine learning workflow. Now we save a cleaned and encoded dataset for further use in ML model.

In [9]:
# Export the cleaned dataframe to a CSV file in the folder for processed CSV
df_cleaned = df.copy()
df_cleaned.to_csv('dataset/processed/telecom_customer_churn_cleaned.csv', index=False)
df_encoded = df_encoded.copy()
df_encoded.to_csv('dataset/processed/telecom_customer_churn_encoded.csv', index=False)

print(f'Cleaned and encoded dataset exported to dataset/processed/')

AttributeError: 'numpy.ndarray' object has no attribute 'to_csv'

---

To be contined in Section 3 for Data Visualization.